<a href="https://colab.research.google.com/github/Tarikol-Islam/Lifespan-Age-Transformation/blob/main/Lifespan_Age_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

In [2]:
!python download_models.py

  0% 3.10k/213M [00:00<2:49:03, 21.0kB/s]Google Drive download failed.
Trying do download from alternate server

  0% 0.00/213M [00:00<?, ?B/s]
  0% 0.00/213M [00:00<?, ?B/s]
  0% 131k/213M [00:00<13:05, 271kB/s]
  0% 393k/213M [00:00<09:42, 365kB/s]
  0% 1.05M/213M [00:00<06:59, 506kB/s]
  1% 1.70M/213M [00:00<05:05, 691kB/s]
  1% 2.23M/213M [00:01<03:49, 921kB/s]
  1% 2.62M/213M [00:01<03:03, 1.15MB/s]
  2% 3.28M/213M [00:01<02:21, 1.49MB/s]
  2% 3.67M/213M [00:01<01:58, 1.76MB/s]
  2% 4.06M/213M [00:01<01:45, 1.98MB/s]
  2% 4.46M/213M [00:01<01:42, 2.04MB/s]
  2% 4.85M/213M [00:01<01:35, 2.19MB/s]
  2% 5.24M/213M [00:02<01:34, 2.20MB/s]
  3% 5.64M/213M [00:02<01:28, 2.35MB/s]
  3% 6.03M/213M [00:02<01:23, 2.47MB/s]
  3% 6.42M/213M [00:02<01:26, 2.39MB/s]
  3% 6.82M/213M [00:02<01:21, 2.54MB/s]
  3% 7.21M/213M [00:02<01:19, 2.58MB/s]
  4% 7.60M/213M [00:02<01:17, 2.67MB/s]
  4% 8.00M/213M [00:03<01:20, 2.54MB/s]
  4% 8.39M/213M [00:03<01:17, 2.63MB/s]
  4% 8.78M/213M [00:03<01:17, 2.

In [ ]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

In [4]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


AgingDataLoader
dataset [MulticlassUnalignedDataset] was created


In [ ]:

opt.name = 'males_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

In [19]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Saving 8.jpg to 8.jpg
User uploaded file "8.jpg"


In [20]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)
os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)


In [ ]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

!pip3 install webm
webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
!webm -i $out_path $webm_out_path
use_webm = True


In [22]:
from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))




files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>